In [1]:
import vk_api

Предварительный анализ скачанных жсон

In [2]:
jposts = vk_api.load_json('posts.json')

In [3]:
jposts[:5]

[{'attachments': [{'photo': {'access_key': '008739e87d330314d1',
     'album_id': -7,
     'date': 1493664165,
     'height': 750,
     'id': 456239019,
     'owner_id': 23704358,
     'photo_130': 'https://pp.userapi.com/c638720/v638720358/3224b/TSsh8ZWB2oA.jpg',
     'photo_604': 'https://pp.userapi.com/c638720/v638720358/3224c/ZUKVH-KDb30.jpg',
     'photo_75': 'https://pp.userapi.com/c638720/v638720358/3224a/A--QVaIuMAM.jpg',
     'photo_807': 'https://pp.userapi.com/c638720/v638720358/3224d/mN0bMSCNPmw.jpg',
     'post_id': 3560,
     'text': '',
     'width': 750},
    'type': 'photo'}],
  'comments': {'can_post': 0, 'count': 0, 'groups_can_post': True},
  'date': 1493664166,
  'from_id': 23704358,
  'id': 3560,
  'likes': {'can_like': 1, 'can_publish': 1, 'count': 9, 'user_likes': 0},
  'owner_id': 23704358,
  'post_source': {'platform': 'instagram',
   'type': 'api',
   'url': 'https://www.instagram.com/p/BTgQBCqFgOO/'},
  'post_type': 'post',
  'reposts': {'count': 0, 'user_re

In [4]:
jposts[0].keys()

dict_keys(['attachments', 'comments', 'date', 'from_id', 'id', 'likes', 'owner_id', 'post_source', 'post_type', 'reposts', 'text', 'views'])

In [5]:
jposts[1013].keys()

dict_keys(['comments', 'copy_history', 'date', 'from_id', 'id', 'likes', 'owner_id', 'post_source', 'post_type', 'reposts', 'text'])

Трансформация json в dataframe реализовано в модуле vk_api
в двух методах - read_posts и read_users

In [6]:
post_df = vk_api.read_posts()
post_df.columns

Index(['owner_id', 'text', 'comments_count', 'likes_count', 'reposts_count',
       'date', 'post_type', 'views_count', 'copy_history_text',
       'copy_history_post_type', 'attachments_video_count',
       'attachments_photo_count', 'attachments_audio_count'],
      dtype='object')

In [7]:
user_df = vk_api.read_users()
print(user_df.columns)
user_df = user_df[['id','about', 'byear', 'books', 'activities', 'graduation',  'interests','movies', 'music', 'sex', 'relation' ]]

Index(['about', 'activities', 'bdate', 'books', 'byear', 'country_id',
       'country_title', 'education_form', 'education_status', 'faculty',
       'faculty_name', 'first_name', 'graduation', 'home_phone', 'home_town',
       'id', 'interests', 'last_name', 'mobile_phone', 'movies', 'music',
       'personal_alcohol', 'personal_inspired_by', 'personal_langs_0',
       'personal_langs_1', 'personal_langs_10', 'personal_langs_11',
       'personal_langs_12', 'personal_langs_13', 'personal_langs_14',
       'personal_langs_15', 'personal_langs_16', 'personal_langs_17',
       'personal_langs_18', 'personal_langs_19', 'personal_langs_2',
       'personal_langs_20', 'personal_langs_21', 'personal_langs_22',
       'personal_langs_23', 'personal_langs_24', 'personal_langs_25',
       'personal_langs_26', 'personal_langs_27', 'personal_langs_28',
       'personal_langs_29', 'personal_langs_3', 'personal_langs_4',
       'personal_langs_5', 'personal_langs_6', 'personal_langs_7',
       'pe

Итого определились с полями у пользователей и в постах

In [8]:
for col in post_df.columns:
    print(col)

owner_id
text
comments_count
likes_count
reposts_count
date
post_type
views_count
copy_history_text
copy_history_post_type
attachments_video_count
attachments_photo_count
attachments_audio_count


In [9]:
for col in user_df.columns:
    print(col)    

id
about
byear
books
activities
graduation
interests
movies
music
sex
relation


Заполним отсутствующий текст текстом из репоста, а если вообще нет текстат ни там ни там, заполним типом поста - есть варианты можно заполнить типа вложений

In [10]:
post_df.loc[post_df['text']=='','text'] = post_df['copy_history_text'].fillna(post_df['post_type'])

Удалим дубликаты

In [11]:
post_df = post_df.drop_duplicates()
post_df.count()

owner_id                   29929
text                       29929
comments_count             29929
likes_count                29929
reposts_count              29929
date                       29929
post_type                  29929
views_count                29929
copy_history_text          22236
copy_history_post_type     22236
attachments_video_count     6035
attachments_photo_count     6035
attachments_audio_count     6035
dtype: int64

Смержим с пользователями

In [12]:
post_df = post_df.merge(user_df, how="inner", left_on="owner_id",right_on='id')

Пермешаем посты  и для начала нам достаточно года рождения и текста, с них и начнем

In [13]:
#post_df
from sklearn.utils import shuffle
df = shuffle(post_df[['byear', 'text']])

In [14]:
df

,byear,text
28653,1996,Смешная озвучка фильма ТОР
7219,1983,Семь «тайных» садов и скверов Петербурга \n \n...
6748,1994,Туалет это реально место реальных людей))))аха...
6957,1974,Быстрый тортик - Вы такой еще не пробовали!\n\...
15997,1968,Время Успешных МАМ!
21299,1988,Этот сезон станет последним для Роналдиньо 😢 😰...
28573,1996,"Я хочу творить добро. Я хочу, чтобы мир стал л..."
15638,1992,
16863,1991,
28243,1986,"""От большой любви рождаются красивые детки..."""


Разобьем на train и test

In [15]:
df_train = df[0:10000]
df_test = df[10001:20000]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
x = count_vect.fit_transform(df_train['text']).todense()

In [17]:
x.shape

(10000, 70768)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(df_train['text'])

In [19]:
features.shape

(10000, 70768)

In [20]:
vectorizer.stop_words_

set()